## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

pubdate source  \
0  2025-10-22-08-48-00 +0000    wsj   
1  2025-10-22-08-46-59 +0000    bbc   
2  2025-10-22-08-46-52 +0000    bbc   
3  2025-10-22-08-37-35 +0000    bbc   
4  2025-10-22-08-32-56 +0000    nyt   

                                               title  \
0  U.S. Futures, Global Markets Mixed; Sterling S...   
1         Eurostar orders first double-decker trains   
2  Six dead in Russian airstrikes, hours after Tr...   
3  UK and Scottish governments dispute £26m cost ...   
4  In a Reversal, Trump Will Not Meet With Putin ...   

                                                link  
0  https://www.wsj.com/finance/investing/u-s-futu...  
1  https://www.bbc.com/news/articles/cz6n1w80z1zo...  
2  https://www.bbc.com/news/articles/czjpe0193geo...  
3  https://www.bbc.com/news/articles/c051mzgyq2vo...  
4  https://www.nytimes.com/2025/10/21/us/politics...

## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2318/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


word  score
18   trump     52
279  house     18
278  white     15
305   gaza     14
434     up     14

## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


pubdate  source  \
313  2025-10-21-11-43-48 +0000     cbc   
226  2025-10-21-17-55-57 +0000  nypost   
214  2025-10-21-18-44-04 +0000     bbc   
78   2025-10-22-02-04-22 +0000     nyt   
266  2025-10-21-15-54-55 +0000  nypost   

                                                 title  \
313  White House demolition work begins as Trump pu...   
226  Hillary mocked after blasting Trump’s White Ho...   
214  From bowling alley to ballroom - Trump isn't t...   
78   As Trump Moves Forward With White House Ballro...   
266  White House rips report Trump could commute Di...   

                                                  link  score  
313  https://www.cbc.ca/news/world/us-white-house-d...    107  
226  https://nypost.com/2025/10/21/us-news/hillary-...    107  
214  https://www.bbc.com/news/articles/cr7m2v80dy4o...    107  
78   https://www.nytimes.com/2025/10/21/arts/design...    102  
266  https://nypost.com/2025/10/21/us-news/white-ho...    101

## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


score                    pubdate  source  \
313    107  2025-10-21-11-43-48 +0000     cbc   
69      76  2025-10-22-03-11-48 +0000     nyt   
112     47  2025-10-21-23-38-00 +0000     wsj   
227     43  2025-10-21-17-44-24 +0000  nypost   
293     42  2025-10-21-13-56-46 +0000  nypost   
249     41  2025-10-21-16-39-19 +0000     nyt   
297     40  2025-10-21-13-32-06 +0000     nyt   
244     40  2025-10-21-17-02-40 +0000  nypost   
100     38  2025-10-22-00-24-39 +0000  nypost   
126     38  2025-10-21-23-10-23 +0000  nypost   

                                                 title  \
313  White House demolition work begins as Trump pu...   
69   Vance Lands in Israel as U.S. Tries to Shore U...   
112  Paul Ingrassia withdrew Tuesday from a Senate ...   
227  Hamas released bodies of two more dead hostage...   
293  Warner Bros. Discovery says it’s open to a sal...   
249  France Sends Former President Nicolas Sarkozy ...   
297  Suspect Charged in Paris Museum Heist (No, Not...   
244  Top Trump adviser boldly predicts Democrats wi...   
100  Hunter Biden says watching Obama lead Joe off ...   
126  Netflix shares slide on rare earnings miss — s...   

                                                  link  
313  https://www.cbc.ca/news/world/us-white-house-d...  
69   https://www.nytimes.com/2025/10/21/world/middl...  
112  https://www.wsj.com/politics/policy/paul-ingra...  
227  https://nypost.com/2025/10/21/world-news/hamas...  
293  https://nypost.com/2025/10/21/business/warner-...  
249  https://www.nytimes.com/2025/10/21/world/europ...  
297  https://www.nytimes.com/2025/10/21/world/europ...  
244  https://nypost.com/2025/10/21/us-news/top-trum...  
100  https://nypost.com/2025/10/21/us-news/hunter-b...  
126  https://nypost.com/2025/10/21/media/netflix-sh...

## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
